In [3]:
import datetime
from statistics import mean
from pprint import pprint

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from data import tfrecord_dataset, train_test_split
from train import train_model
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_SMALL_UNLABELED_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
    SEED,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [4]:
# Hyperparameters
val_perc = 0.2
batch_size = 16
patience = 30
learning_rate = 1e-5

In [5]:
dataset = tfrecord_dataset(
    [LIDC_SMALL_NEG_TFRECORD, LIDC_SMALL_POS_TFRECORD, LIDC_SMALL_UNLABELED_TFRECORD]
)
dataset = tf.data.Dataset.zip((dataset, dataset))
num_samples = sum(1 for _ in dataset)
num_samples

1139

In [6]:
def build_model():
    encoder = keras.Sequential(
        [
            keras.layers.InputLayer(SMALL_PATCH_SHAPE, name="encoder_input"),
            SeluConv3D(filters=32, kernel_size=3, name="encoder_selu_conv3d_1"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_1"),
            SeluConv3D(filters=64, kernel_size=3, name="encoder_selu_conv3d_2"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_2"),
            SeluConv3D(filters=128, kernel_size=3, name="encoder_selu_conv3d_3"),
            keras.layers.MaxPooling3D((1, 2, 2), name="maxpool_3"),
            SeluConv3D(filters=256, kernel_size=3, name="encoder_selu_conv3d_4"),
        ],
        name="encoder",
    )
    decoder = keras.Sequential(
        [
            keras.layers.InputLayer(encoder.output_shape[1:], name="decoder_input"),
            SeluConv3D(filters=256, kernel_size=3, name="decoder_selu_conv3d_1"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_2"),
            SeluConv3D(filters=128, kernel_size=3, name="decoder_selu_conv3d_2"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_3"),
            SeluConv3D(filters=64, kernel_size=3, name="decoder_selu_conv3d_3"),
            keras.layers.UpSampling3D((1, 2, 2), name="upsampling_4"),
            SeluConv3D(filters=32, kernel_size=3, name="decoder_selu_conv3d_4"),
            keras.layers.Dense(1, activation="sigmoid", name="decoder_final_dense"),
        ],
        name="decoder",
    )

    autoencoder = keras.Sequential(
        [
            keras.layers.InputLayer(SMALL_PATCH_SHAPE, name="autoencoder_input"),
            encoder,
            decoder,
        ],
        name="autoencoder",
    )
    return autoencoder

In [7]:
train_dataset, val_dataset = train_test_split(
    dataset,
    test_perc=val_perc,
    cardinality=num_samples,
)
val_dataset = val_dataset.batch(1).cache().prefetch(tf.data.experimental.AUTOTUNE)
train_dataset = (
    train_dataset.batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=64, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

autoencoder = build_model()
autoencoder.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.MeanSquaredError(),
)

model_fname = f"models/autoencoder-lidc.h5"
log_dir = f"logs/autoencoder-lidc"
autoencoder = train_model(
    autoencoder,
    train_dataset,
    val_dataset,
    patience,
    "val_loss",
    model_fname,
    log_dir,
    verbose_checkpoint=1,
)


Epoch 00001: val_loss improved from inf to 0.05111, saving model to models/autoencoder-lidc.h5

Epoch 00002: val_loss improved from 0.05111 to 0.04566, saving model to models/autoencoder-lidc.h5

Epoch 00003: val_loss improved from 0.04566 to 0.04312, saving model to models/autoencoder-lidc.h5

Epoch 00004: val_loss improved from 0.04312 to 0.04160, saving model to models/autoencoder-lidc.h5

Epoch 00005: val_loss improved from 0.04160 to 0.04060, saving model to models/autoencoder-lidc.h5

Epoch 00006: val_loss improved from 0.04060 to 0.03991, saving model to models/autoencoder-lidc.h5

Epoch 00007: val_loss improved from 0.03991 to 0.03931, saving model to models/autoencoder-lidc.h5

Epoch 00008: val_loss improved from 0.03931 to 0.03880, saving model to models/autoencoder-lidc.h5

Epoch 00009: val_loss improved from 0.03880 to 0.03859, saving model to models/autoencoder-lidc.h5

Epoch 00010: val_loss improved from 0.03859 to 0.03817, saving model to models/autoencoder-lidc.h5

Epo


Epoch 00095: val_loss improved from 0.03429 to 0.03428, saving model to models/autoencoder-lidc.h5

Epoch 00096: val_loss improved from 0.03428 to 0.03425, saving model to models/autoencoder-lidc.h5

Epoch 00097: val_loss did not improve from 0.03425

Epoch 00098: val_loss improved from 0.03425 to 0.03421, saving model to models/autoencoder-lidc.h5

Epoch 00099: val_loss did not improve from 0.03421

Epoch 00100: val_loss improved from 0.03421 to 0.03419, saving model to models/autoencoder-lidc.h5

Epoch 00101: val_loss improved from 0.03419 to 0.03419, saving model to models/autoencoder-lidc.h5

Epoch 00102: val_loss improved from 0.03419 to 0.03417, saving model to models/autoencoder-lidc.h5

Epoch 00103: val_loss did not improve from 0.03417

Epoch 00104: val_loss did not improve from 0.03417

Epoch 00105: val_loss improved from 0.03417 to 0.03413, saving model to models/autoencoder-lidc.h5

Epoch 00106: val_loss did not improve from 0.03413

Epoch 00107: val_loss improved from 0.0


Epoch 00205: val_loss improved from 0.03341 to 0.03340, saving model to models/autoencoder-lidc.h5

Epoch 00206: val_loss improved from 0.03340 to 0.03339, saving model to models/autoencoder-lidc.h5

Epoch 00207: val_loss improved from 0.03339 to 0.03339, saving model to models/autoencoder-lidc.h5

Epoch 00208: val_loss did not improve from 0.03339

Epoch 00209: val_loss did not improve from 0.03339

Epoch 00210: val_loss did not improve from 0.03339

Epoch 00211: val_loss improved from 0.03339 to 0.03338, saving model to models/autoencoder-lidc.h5

Epoch 00212: val_loss improved from 0.03338 to 0.03337, saving model to models/autoencoder-lidc.h5

Epoch 00213: val_loss did not improve from 0.03337

Epoch 00214: val_loss did not improve from 0.03337

Epoch 00215: val_loss improved from 0.03337 to 0.03336, saving model to models/autoencoder-lidc.h5

Epoch 00216: val_loss did not improve from 0.03336

Epoch 00217: val_loss improved from 0.03336 to 0.03336, saving model to models/autoenco


Epoch 00317: val_loss did not improve from 0.03309

Epoch 00318: val_loss improved from 0.03309 to 0.03309, saving model to models/autoencoder-lidc.h5

Epoch 00319: val_loss did not improve from 0.03309

Epoch 00320: val_loss improved from 0.03309 to 0.03309, saving model to models/autoencoder-lidc.h5

Epoch 00321: val_loss did not improve from 0.03309

Epoch 00322: val_loss did not improve from 0.03309

Epoch 00323: val_loss did not improve from 0.03309

Epoch 00324: val_loss did not improve from 0.03309

Epoch 00325: val_loss did not improve from 0.03309

Epoch 00326: val_loss did not improve from 0.03309

Epoch 00327: val_loss did not improve from 0.03309

Epoch 00328: val_loss did not improve from 0.03309

Epoch 00329: val_loss improved from 0.03309 to 0.03306, saving model to models/autoencoder-lidc.h5

Epoch 00330: val_loss did not improve from 0.03306

Epoch 00331: val_loss did not improve from 0.03306

Epoch 00332: val_loss did not improve from 0.03306

Epoch 00333: val_loss d


Epoch 00446: val_loss did not improve from 0.03292

Epoch 00447: val_loss did not improve from 0.03292

Epoch 00448: val_loss improved from 0.03292 to 0.03292, saving model to models/autoencoder-lidc.h5

Epoch 00449: val_loss did not improve from 0.03292

Epoch 00450: val_loss did not improve from 0.03292

Epoch 00451: val_loss did not improve from 0.03292

Epoch 00452: val_loss did not improve from 0.03292

Epoch 00453: val_loss did not improve from 0.03292

Epoch 00454: val_loss did not improve from 0.03292

Epoch 00455: val_loss did not improve from 0.03292

Epoch 00456: val_loss did not improve from 0.03292

Epoch 00457: val_loss did not improve from 0.03292

Epoch 00458: val_loss did not improve from 0.03292

Epoch 00459: val_loss improved from 0.03292 to 0.03290, saving model to models/autoencoder-lidc.h5

Epoch 00460: val_loss did not improve from 0.03290

Epoch 00461: val_loss did not improve from 0.03290

Epoch 00462: val_loss did not improve from 0.03290

Epoch 00463: val_lo


Epoch 00587: val_loss did not improve from 0.03284

Epoch 00588: val_loss did not improve from 0.03284

Epoch 00589: val_loss did not improve from 0.03284

Epoch 00590: val_loss did not improve from 0.03284

Epoch 00591: val_loss did not improve from 0.03284

Epoch 00592: val_loss did not improve from 0.03284

Epoch 00593: val_loss did not improve from 0.03284

Epoch 00594: val_loss did not improve from 0.03284

Epoch 00595: val_loss did not improve from 0.03284

Epoch 00596: val_loss did not improve from 0.03284

Epoch 00597: val_loss did not improve from 0.03284

Epoch 00598: val_loss did not improve from 0.03284

Epoch 00599: val_loss improved from 0.03284 to 0.03283, saving model to models/autoencoder-lidc.h5

Epoch 00600: val_loss did not improve from 0.03283

Epoch 00601: val_loss did not improve from 0.03283

Epoch 00602: val_loss did not improve from 0.03283

Epoch 00603: val_loss did not improve from 0.03283

Epoch 00604: val_loss did not improve from 0.03283

Epoch 00605: va

In [9]:
original, _ = next(iter(val_dataset.skip(10)))
encoder_out = autoencoder.get_layer("encoder")(original, training=False)
decoder_out = autoencoder.get_layer("decoder")(encoder_out, training=False)
plot_volume_animation(original[0, :])

In [ ]:
batch_index = 0
z_index = 3
fig, ax = plt.subplots(ncols=3)
plot_slice(original[batch_index, :], index=z_index, ax=ax[0])
plot_slice(encoder_out[batch_index, :], index=z_index, ax=ax[1])
plot_slice(decoder_out[batch_index, :], index=z_index, ax=ax[2])

In [ ]:
plot_animated_volume(original[0, :])